This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [1]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *
#from nltk.sem.drt import DrtParser

from nltk.sem.logic import *
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## Axioms

### Lattice axioms

In [2]:
part_def = read_expr('Partof(x,y) <-> Product(x, y) = x')
proper_part_def = read_expr('Properpartof(x,y) <-> Partof(x,y) & -(x=y)')

sum_id = read_expr('Sum(x,x) = x')
product_id = read_expr('Product(x,x) = x')

sum_comm = read_expr('Sum(x,y) = Sum(y,x)')
product_comm = read_expr('Product(x,y) = Product(y,x)')

sum_assoc = read_expr('Sum(x,Sum(y,z)) = Sum(Sum(x,y),z)')
product_assoc = read_expr('Product(x,Product(y,z)) = Product(Product(x,y),z)')

sum_absorp = read_expr('Sum(x,Product(x,y)) = x')
product_absorp = read_expr('Product(x,Sum(x,y)) = x')

lattice = [part_def, proper_part_def, sum_id, product_id, sum_comm, product_comm, sum_assoc, product_assoc, sum_absorp, product_absorp]

### Bounded lattice axioms

In [3]:
bot = read_expr('Sum(x, Bot) = x')
top = read_expr('Product(x, Top) = x')

bounded_lattice = lattice + [bot, top]

### Distributive lattice axioms

In [4]:
dist = read_expr('Product(x,Sum(y,z)) = Sum(Product(x,y), Product(x,z))')

dist_bounded_lattice = bounded_lattice + [dist]

### Complete lattice axioms

In [5]:
uniq_sum = read_expr('all P.(exists z.((all x.(P(x) -> Partof(x,z))) & all y.((all x.(P(x) -> Partof(x,y))) -> Partof(z,y))))')
uniq_product = read_expr('all P.(exists z.((all x.(P(x) -> Partof(z,x))) & all y.((all x.(P(x) -> Partof(y,x))) -> Partof(y,z))))')
# this only guarantees existence and not uniqueness, 
# but sums and products are unique wherever they exist

### Boolean algebra axioms

In [6]:
complemented = read_expr('all x.(exists y. (Product(x, y) = Bot & Sum(x, y) = Top))')

#sectional complement
sec_compl = read_expr('all x. (all y. (exists z. (Sum(x, z) = y & Product(x, z) = Bot)))')

boolean_algebra = dist_bounded_lattice + [uniq_sum, uniq_product, complemented]

### Exclusion axioms

In [7]:
excl_cum = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(Sum(x1, x2), Sum(y1, y2))')
excl_symm = read_expr('Excl(x, y) <-> Excl(y, x)')

### Possibility axioms

In [15]:
#define conflict:
confl_def = read_expr('Confl(x, y) <-> exists z1.(Partof(z1,x) & exists z2.(Partof(z2,y) & Excl(z1, z2)))')

#define possibilities:
#int_poss_def = read_expr('int_poss(x) <-> -Excl(x, x)')
#ext_poss_def = read_expr('ext_poss(x) <-> exists y.(int_poss(y) & -exists z.(Product(y, z) = y & int_poss(z) & -(confl(z,x))))')

poss_def = read_expr('Poss(x) <-> -Confl(x, x)')

#define world:
#world_def = read_expr('world(x) <-> (int_poss(x) & all y.((Product(x, y) = y) -> -int_poss(y)))')

world_def = read_expr('World(x) <-> (Poss(x) & all y.(Properpartof(x,y) -> -Poss(y)))')

#actual world
actual_world = read_expr('World(Actual_world)')

#define actual:
actual_def = read_expr('Actual(x) <-> Partof(x, Actual_world)')

#Harmony
#poss_harm = read_expr('all x.(-int_poss(x) -> -ext_poss(x))')

poss_harm = read_expr('all x.(all y. ((World(x) & -Confl(x, y)) -> Poss(y)))')

#Nirvana
poss_nirv = read_expr('all x.(-Confl(Bot, x))')

#Rashōmon
#poss_rshm = read_expr('all x. (all y.((int_poss(x) & int_poss(y) & -confl(x, y)) -> int_poss(Product(x,y))))')
poss_rshm = read_expr('all x.(all y.((Poss(x) & Poss(y) & -Confl(x, y)) -> Poss(Sum(x,y))))')


#Cosmopolitanism
#poss_cosm = read_expr('all x.(int_poss(x) -> exists y.(world(y) & Product(x, y) = y))')

poss_cosm = read_expr('all x.(Poss(x) -> exists y.(World(y) & Partof(x,y)))')

poss_ax = [poss_harm, poss_nirv, poss_rshm, poss_cosm]

#Construct event frame
#choice point: definition of possibility
#choice point: formulation of harmony

event_space = boolean_algebra + [excl_cum, excl_symm, confl_def, poss_def, world_def, actual_world, actual_def]

event_frame = event_space + poss_ax

In [16]:
#using prover9 to find a proof
def try_to_derive(axiom):
    goal = axiom
    prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == axiom)], timeout = 1000)
    prover.prove()
    print(prover.proof())

#using mace4 to find a counterexample
def try_to_show_independence(axiom):
    goal = axiom
    mb = MaceCommand(goal, assumptions = [ax for ax in event_frame if not (ax == axiom)])
    print(mb.build_model())
    print(mb.model(format='cooked'))

## Independence of axioms

In [53]:
try_to_derive(poss_harm) # times out

KeyboardInterrupt: 

In [17]:
try_to_show_independence(poss_harm) # succeeds

True
% number = 1
% seconds = 0

% Interpretation of size 2

Actual_world = 0.

Bot = 0.

Top = 1.

c1 = 0.

c2 = 1.

f1(0) = 0.
f1(1) = 0.

f3(0) = 0.
f3(1) = 0.

f5(0) = 1.
f5(1) = 0.

f8(0) = 0.
f8(1) = 0.

f9(0) = 0.
f9(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f4(0,0) = 0.
f4(0,1) = 0.
f4(1,0) = 0.
f4(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 0.
f6(1,1) = 1.

f7(0,0) = 0.
f7(0,1) = 0.
f7(1,0) = 0.
f7(1,1) = 1.

  Actual(0).
- Actual(1).

  P(0).
- P(1).

  Poss(0).
- Poss(1).

  World(0).
- World(1).

- Confl(0,0).
- Confl(0,1).
- Confl(1,0).
  Confl(1,1).

- Excl(0,0).
- Excl(0,1).
- Excl(1,0).
  Excl(1,1).

  Partof(0,0).
  Partof(0,1).
- Partof(1,0).
  Partof(1,1).

- Properpartof(0,0).
  Properpartof(0,1).
- Properpartof(1,0).
- Properpartof(1,1).



In [ ]:
try_to_derive(poss_nirv) #fail

In [18]:
try_to_show_independence(poss_nirv) #success

True
% number = 1
% seconds = 0

% Interpretation of size 2

Actual_world = 0.

Bot = 0.

Top = 1.

c1 = 1.

f1(0) = 0.
f1(1) = 0.

f3(0) = 0.
f3(1) = 0.

f5(0) = 1.
f5(1) = 0.

f8(0) = 0.
f8(1) = 0.

f9(0) = 0.
f9(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f4(0,0) = 0.
f4(0,1) = 0.
f4(1,0) = 0.
f4(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 1.
f6(1,1) = 0.

f7(0,0) = 0.
f7(0,1) = 1.
f7(1,0) = 0.
f7(1,1) = 1.

  Actual(0).
- Actual(1).

  P(0).
- P(1).

  Poss(0).
- Poss(1).

  World(0).
- World(1).

- Confl(0,0).
  Confl(0,1).
  Confl(1,0).
  Confl(1,1).

- Excl(0,0).
  Excl(0,1).
  Excl(1,0).
  Excl(1,1).

  Partof(0,0).
  Partof(0,1).
- Partof(1,0).
  Partof(1,1).

- Properpartof(0,0).
  Properpartof(0,1).
- Properpartof(1,0).
- Properpartof(1,1).



In [ ]:
try_to_derive(poss_rshm)

In [19]:
try_to_show_independence(poss_rshm) #success

True
% number = 1
% seconds = 0

% Interpretation of size 4

Actual_world = 0.

Bot = 1.

Top = 2.

c1 = 0.

c2 = 3.

f1(0) = 0.
f1(1) = 0.
f1(2) = 0.
f1(3) = 0.

f3(0) = 0.
f3(1) = 0.
f3(2) = 0.
f3(3) = 0.

f5(0) = 3.
f5(1) = 2.
f5(2) = 1.
f5(3) = 0.

f8(0) = 0.
f8(1) = 0.
f8(2) = 0.
f8(3) = 0.

f9(0) = 0.
f9(1) = 0.
f9(2) = 0.
f9(3) = 3.

Product(0,0) = 0.
Product(0,1) = 1.
Product(0,2) = 0.
Product(0,3) = 1.
Product(1,0) = 1.
Product(1,1) = 1.
Product(1,2) = 1.
Product(1,3) = 1.
Product(2,0) = 0.
Product(2,1) = 1.
Product(2,2) = 2.
Product(2,3) = 3.
Product(3,0) = 1.
Product(3,1) = 1.
Product(3,2) = 3.
Product(3,3) = 3.

Sum(0,0) = 0.
Sum(0,1) = 0.
Sum(0,2) = 2.
Sum(0,3) = 2.
Sum(1,0) = 0.
Sum(1,1) = 1.
Sum(1,2) = 2.
Sum(1,3) = 3.
Sum(2,0) = 2.
Sum(2,1) = 2.
Sum(2,2) = 2.
Sum(2,3) = 2.
Sum(3,0) = 2.
Sum(3,1) = 3.
Sum(3,2) = 2.
Sum(3,3) = 3.

f2(0,0) = 0.
f2(0,1) = 0.
f2(0,2) = 0.
f2(0,3) = 0.
f2(1,0) = 0.
f2(1,1) = 0.
f2(1,2) = 0.
f2(1,3) = 0.
f2(2,0) = 0.
f2(2,1) = 0.
f2(2,2) = 0.


In [60]:
try_to_show_independence(poss_cosm) #timeout (only counterexamples are infinite models?)

KeyboardInterrupt: 

In [13]:
try_to_derive(poss_cosm) #timeout

KeyboardInterrupt: 

## Theorems

### Plenitude

In [20]:
#check plenitude
#goal = read_expr('all x.(int_poss(x) -> ext_poss(x))')

goal = read_expr('all x.((exists y.(-Confl(x, y) & World(y))) <-> Poss(x))')

#goal = read_expr('all x.(Poss(x) <-> (exists y.(-Confl(x, y) & World(y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 12031 was started by guzzler on oahzzdepbm,
Wed Jul 13 17:40:02 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 8.60 (+ 0.38) seconds.
% Length of proof is 56.
% Level of proof is 13.
% Maximum clause weight is 15.000.
% Given clauses 8303.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
7 Excl(x,y) <-> Excl(y,x).  [assumption].
8 Confl(x,y) <-> (exists z (Partof(z,x) & (exists u (Partof(u,y) & Excl(z,u))))).  [assumption].
9 Poss(x) <-> -Confl(x,x).  [assumption].
10 World(x) <-> Poss(x) & (all y (Properpartof(x,y) -> -Poss(y))).  [assumption].
12 (all x all y (World(x) & -Confl(x,y) -

### Distributivity of actuality

In [49]:
#check distributivity of actuality
goal = read_expr('all x.((exists y. Actual(y) & Partof(x,y)) -> Actual(x))')
prover = Prover9Command(goal, assumptions = event_frame, timeout = 1000)
prover.prove()
print(prover.proof())

KeyboardInterrupt: 

In [48]:
#Find counterexample for distributivity of actuality - fail
goal = read_expr('all x.((exists y. (Actual(y) & Partof(x, y))) -> Actual(x))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

False

### Cumulativity of actuality

In [ ]:
#check cumulativity of actuality
goal = read_expr('all P.((all x.(P(x) -> Actual(x))) -> (exists z.((all x.(P(x) -> Partof(x, z))) & (all y.((all x.(P(x) -> Partof(x, y))) -> Partof(z, y)))) & Actual(z)))')
prover = Prover9Command(goal, assumptions = event_frame, timeout = 1000)
prover.prove()
print(prover.proof())

In [70]:
#find counterexample for cumulativity of actuality - fail
goal = read_expr('all P.((all x.(P(x) -> Actual(x))) -> (exists z.(all x.(P(x) -> Product(x, z) = x) & all y.((all x.(P(x) -> Product(x, y) = x)) -> Product(y,z) = z)) & Actual(z)))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

KeyboardInterrupt: 

### Manichaeism

In [24]:
#check Manichaeism
goal = read_expr('all x.((World(x) -> (all y.(Partof(y,x)|Confl(x, y)))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 12074 was started by guzzler on oahzzdepbm,
Wed Jul 13 17:41:55 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.06 (+ 0.01) seconds.
% Length of proof is 30.
% Level of proof is 7.
% Maximum clause weight is 11.000.
% Given clauses 439.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
2 Properpartof(x,y) <-> Partof(x,y) & x != y.  [assumption].
10 World(x) <-> Poss(x) & (all y (Properpartof(x,y) -> -Poss(y))).  [assumption].
12 (all x all y (World(x) & -Confl(x,y) -> Poss(y))).  [assumption].
14 (all x all y (Poss(x) & Poss(y) & -Confl(x,y) -> Poss(Sum(x,y)))).  [assumption].
16 (all x

### Classicality of conjunction

In [25]:
#check classicality of conjunction
goal = read_expr(r'(((exists x. (P(x) & Actual(x))) & (exists y. (Q(y) & Actual(y)))) <-> exists y exists z.(Actual(y) & Actual(z) & P(y) & Q(z) & Actual(Sum(y, z))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 12076 was started by guzzler on oahzzdepbm,
Wed Jul 13 17:42:00 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.28 (+ 0.02) seconds.
% Length of proof is 79.
% Level of proof is 17.
% Maximum clause weight is 20.000.
% Given clauses 1334.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
11 Actual(x) <-> Partof(x,Actual_world).  [assumption].
16 (exists x (P(x) & Actual(x))) & (exists y (Q(y) & Actual(y))) <-> (exists y exists z (Actual(y) & Actual(z) & P(y) & Q(z) & Actual(Sum(y,z)))).  [goal].
30 -Partof(x,y) | Product(x,y) = x.  [clausify(1)].
31 Partof(x,y) | Product(x,y) != x.  [cl

In [21]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & Actual(x))) & (exists y. (Q(y) & Actual(y)))) <-> exists y exists z.(Actual(y) & Actual(z) & P(y) & Q(z) & Actual(Sum(y, z)))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

KeyboardInterrupt: 

### Classicality of disjunction

In [68]:
#check classicality of disjunction
goal = read_expr(r'(exists x. (Actual(x) & (P(x) | Q(x)))) <-> ((exists x. (Actual(x) & P(x))) | (exists y. (Actual(y) & Q(y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 12940 was started by guzzler on oahzzdepbm,
Wed Jul 13 18:27:18 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 38.
% Level of proof is 14.
% Maximum clause weight is 8.000.
% Given clauses 160.


16 (exists x (Actual(x) & (P(x) | Q(x)))) <-> (exists x (Actual(x) & P(x))) | (exists y (Actual(y) & Q(y))).  [goal].
67 Actual(c1) | Actual(c2) | Actual(c3).  [deny(16)].
68 Actual(c1) | Actual(c2) | Q(c3).  [deny(16)].
69 Actual(c1) | P(c2) | Actual(c3).  [deny(16)].
70 Actual(c1) | P(c2) | Q(c3).  [deny(16)].
71 P(c1) | Q(c1) | Actual(c2) | Actual(c3).

In [25]:
goal = read_expr(r'(exists x. (Actual(x) & (P(x) | Q(x)))) <-> ((exists x. (Actual(x) & P(x))) | (exists y. (Actual(y) & Q(y))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

KeyboardInterrupt: 

### No Gaps

In [60]:
#define negating a set
#x negates P iff there is a function h such that x = ⊔{h(xi) ∣ xi ∈ P} and for all events 
# x1 ∈ P, h(x1) excludes some part of x1.
# 
# x negates P iff every x1 in P has a part y that excludes some part x2 of x and
# every part x3 of x

# def negate(x, P):
#     result = ('(all x1. (P(x1) -> (exists x2. Partof(x2,x) & exists y. Partof(y,x1) & Excl(x2, y)))) 
#               & (all x3. ((Partof(x3,x) & (-exists x4. (Partof(x4,x) & Partof(x4,x3)))) -> (exists x5. (P(x5) & Excl(x5, x3))))) & all x6 all x7. ((x6 != x7) -> (exists x8 exists x9.(Partof(x8,x) & Partof(x9,x) & x8!=x9 & Excl(x6, x8) & Excl(x7, x9))))')
#     return result


#define exclusionary negation: an event x exclusionarily negates P iff
#there is a set of events Q such that for each event x in P, part of x is excluded by some event y in Q
#and each event y in Q excludes some part of an event x in P
#and x is the sum of Q.
def exclusionary_negate(x, P):
    result = '(exists Q. ((all x1. (' + P + '(x1) -> (exists x2. Partof(x2, x1) & (exists y1. (Q(y1) & Excl(y1, x2))))))'\
             '& (all y2. (Q(y2) -> (exists x3. ('+P+'(x3) & (exists x4. (Partof(x4, x3) & Excl(y2, x4)))))))'\
              '& (all y3. (Q(y3) -> Partof(y3,'+x+'))) & (all y5. ((all y4.(Q(y4) -> Partof(y4, y5))) -> Partof('+x+', y5)))))'
    return result

#define preclude a set: an event x precludes P iff there exists a Q such that
#for any x1 in P, there is y1 in Q such that y1 excludes part of x1
#for any y2 in Q, there is x3 in P such that y2 excludes part of x3
#for any distinct y3, y4 in Q, there are distinct x5, x6 such that y3 excludes part of x5, y4 excludes part of x6
#x is the sum of Q
def preclude(x, P):
    result = '(exists Q. ((all x1.('+P+'(x1) -> (exists y1. (Q(y1) & (exists x2. (Partof(x2, x1) & Excl(y1, x2)))))))'\
             '& (all y2. (Q(y2) -> (exists x3. ('+P+'(x3) & (exists x4. (Partof(x4, x3) & Excl(y2, x4)))))))'\
             '& (all y3. (all y4. ((Q(y3) & Q(y4) & y3 != y4) -> (exists x5. (exists x6. ('+P+'(x5) & '+P+'(x6) & exists x7. (exists x8. (Partof(x7, x5) & (Partof(x8, x6) & Excl(y3, x7) & Excl(y4, x8))))))))))'\
             '& ((all y5. (Q(y5) -> Partof(y5, '+x+'))) & (all y6. ((all y7. (Q(y7) -> Partof(y7, y6))) -> Partof('+x+', y6))))))'
    return result

In [61]:
#check no gaps with preclusion-based negation
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
prover = Prover9Command(goal, assumptions = event_frame, timeout=1000)
prover.prove()
print(prover.proof())

KeyboardInterrupt: 

In [65]:
goal = read_expr(r'all P. (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+')))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))

% number = 1
% seconds = 0

% Interpretation of size 2

Actual_world = 0.

Bot = 1.

Top = 0.

c2 = 0.

f1(0) = 1.
f1(1) = 1.

f3(0) = 0.
f3(1) = 0.

f5(0) = 1.
f5(1) = 0.

f8(0) = 0.
f8(1) = 0.

f9(0) = 0.
f9(1) = 0.

Product(0,0) = 0.
Product(0,1) = 1.
Product(1,0) = 1.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 0.
Sum(1,0) = 0.
Sum(1,1) = 1.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f4(0,0) = 0.
f4(0,1) = 0.
f4(1,0) = 0.
f4(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 0.
f6(1,1) = 0.

f7(0,0) = 0.
f7(0,1) = 0.
f7(1,0) = 0.
f7(1,1) = 0.

f10(0,0) = 0.
f10(0,1) = 0.
f10(1,0) = 0.
f10(1,1) = 0.

f11(0,0) = 0.
f11(0,1) = 0.
f11(1,0) = 0.
f11(1,1) = 0.

f12(0,0) = 0.
f12(0,1) = 0.
f12(1,0) = 0.
f12(1,1) = 0.

f13(0,0) = 0.
f13(0,1) = 0.
f13(1,0) = 0.
f13(1,1) = 0.

f14(0,0) = 0.
f14(0,1) = 0.
f14(1,0) = 0.
f14(1,1) = 0.

f15(0,0) = 0.
f15(0,1) = 0.
f15(1,0) = 0.
f15(1,1) = 0.

  Actual(0).
  Actual(1).

- P(0).
- P(1).

  Poss(0).
  Poss(1).

  Q(0).
- Q(1).

  World(0).
-

In [47]:
#check no gaps with preclusion-based negation
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
prover = Prover9Command(goal, assumptions = event_frame, timeout=1000)
prover.prove()
print(prover.proof())

KeyboardInterrupt: 

In [44]:
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

False

### No Gluts

In [37]:
#check no gluts with exclusionary negation
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+exclusionary_negate('z', 'P')+')))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

KeyboardInterrupt: 

In [66]:
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+exclusionary_negate('z', 'P')+')))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Actual_world = 0.

Bot = 1.

Top = 2.

c1 = 2.

c3 = 0.

c4 = 0.

c5 = 0.

f1(0) = 2.
f1(1) = 2.
f1(2) = 2.
f1(3) = 2.

f3(0) = 0.
f3(1) = 0.
f3(2) = 0.
f3(3) = 0.

f5(0) = 3.
f5(1) = 2.
f5(2) = 1.
f5(3) = 0.

f8(0) = 0.
f8(1) = 0.
f8(2) = 0.
f8(3) = 0.

f9(0) = 0.
f9(1) = 0.
f9(2) = 0.
f9(3) = 0.

f10(0) = 0.
f10(1) = 0.
f10(2) = 0.
f10(3) = 0.

f11(0) = 0.
f11(1) = 0.
f11(2) = 0.
f11(3) = 0.

f12(0) = 2.
f12(1) = 0.
f12(2) = 0.
f12(3) = 0.

f13(0) = 2.
f13(1) = 0.
f13(2) = 0.
f13(3) = 0.

f14(0) = 0.
f14(1) = 0.
f14(2) = 0.
f14(3) = 0.

Product(0,0) = 0.
Product(0,1) = 1.
Product(0,2) = 0.
Product(0,3) = 1.
Product(1,0) = 1.
Product(1,1) = 1.
Product(1,2) = 1.
Product(1,3) = 1.
Product(2,0) = 0.
Product(2,1) = 1.
Product(2,2) = 2.
Product(2,3) = 3.
Product(3,0) = 1.
Product(3,1) = 1.
Product(3,2) = 3.
Product(3,3) = 3.

Sum(0,0) = 0.
Sum(0,1) = 0.
Sum(0,2) = 2.
Sum(0,3) = 2.
Sum(1,0) = 0.
Sum(1,1) = 1.
Sum(1,2) = 2.
Sum(1,3) = 3

In [67]:
#check no gluts with preclusion-based negation
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+preclude('z', 'P')+')))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 12935 was started by guzzler on oahzzdepbm,
Wed Jul 13 18:27:07 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.04 (+ 0.00) seconds.
% Length of proof is 46.
% Level of proof is 8.
% Maximum clause weight is 12.000.
% Given clauses 288.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
8 Confl(x,y) <-> (exists z (Partof(z,x) & (exists u (Partof(u,y) & Excl(z,u))))).  [assumption].
9 Poss(x) <-> -Confl(x,x).  [assumption].
10 World(x) <-> Poss(x) & (all y (Properpartof(x,y) -> -Poss(y))).  [assumption].
16 (all P -(exists x (World(x) & (exists y (Partof(y,x) & P(y))) & (exists z (Partof(

In [54]:
goal = read_expr(r'all P. (-exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+preclude('z', 'P')+'))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))

KeyboardInterrupt: 